# 1-單純貝氏 (詩詞分類)

## 應用:
 * 分類垃圾信
 
## 機率:
* P(A) = (A事件)發生的機率
* P(A|B) = $ P(\frac{A}{B}) = \frac{P(A\cap{B})}{P(B)} $ = 如果(B事件)發生,則(A事件)發生的機率
* $ P(A\cap{B}) $ = (A事件)與(B事件)同時發生的機率
* $ P(A\cap{B}) = P(B)P(A|B) $ = 先發生(B事件),再發生(A事件)的機率
* 如果(A事件)與(B事件)相互獨立, 則 $ P(A\cap{B}) = P(A)P(B) $
* 貝葉斯定理: $ P(A\cap{B}) = P(A)P(B|A) = P(B)P(A|B) $

## 詩詞分類
* P(李白) = 任選一首詩,該首詩是李白寫的機率
* P(ABC|李白) = 如果該首詩是李白寫的,則該首詩內容出現(ABC)的機率
* P(李白|ABC) = 如果該首詩裡面有內容(ABC),則該首詩是李白寫的機率
* $ P(ABC\cap{李白}) = $(該首詩的內容出現ABC)且(該首詩是李白寫的)

## 詞彙之間是有順序關係的:
* (A詞彙) 的機率 = $ P(A) $
* 如果先出現(A詞彙),再出現(B詞彙) 的機率 = $ P(A)*P(B|A) $
* 如果先出現(A詞彙),再出現(B詞彙),再出現(C詞彙) 的機率 = $ P(A)*P(B|A)*P(C|A\cap{B}) $

## 該分類目前不考慮詞彙順序:
* 詩的內容: A\B\C
* P(A|李白) = 李白寫出(A)的機率 = $ \frac{A出現的次數}{李白寫出A的次數} $
* P(A|李白)*P(B|李白)*P(C|李白) = 李白寫出(ABC)的機率

# 2-資料集
* poem_train.csv: 供你訓練模型
* poem_test.csv: 供你測試模型
* 標籤:[李白,杜甫,白居易]


 - 問題:維度災難 = 特徵維度過大 --> 特徵太多導致無法進型特徵選擇
 - 解決:降低維度

In [1]:
# 匯入資料
import pandas as pd
train_df = pd.read_csv("./data/dataset_8/poem_train.csv", encoding = "utf-8")
test_df = pd.read_csv("./data/dataset_8/poem_test.csv", encoding = "utf-8")

In [2]:
import jieba
s = """平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r\n何處是歸程？長亭連短亭。"""
s = s.replace("\r\n","").replace("\r\n","")
" ".join(jieba.cut(s))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 1.124 seconds.
Prefix dict has been built succesfully.


'平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 暝 色入 高樓 ， 有人 樓上 愁 。 玉階空 佇立 ， 宿鳥 歸 飛急 。 何處 是 歸程 ？ 長 亭 連短亭 。'

## [decorator](https://dotblogs.com.tw/rickyteng/archive/2013/11/06/126852.aspx)

In [3]:
# 顯示function的名稱
def show(func):
    return func.__name__

print(show(int))

int


In [4]:
def show(func):
    print("step1:")
    print(func.__name__)
    return func

@show
def test2(a):
    print("step2:")
    return a+3

test2(5) # @show --> 先執行 show(test2), 再執行 test2(5)

step1:
test2
step2:


8

In [5]:
def show(func):
    print("show")
    if func.__name__=="test2":
        print("show_A")
        return func
    else:
        print("show_B")
        return int

@show
def test(a):
    print("test")
    return a+5

@show
def test2(a):
    print("test2")
    return a+3

# test(5) # test --> show_B --> show --> show_A
# print("---------")
# test2(5)

show
show_B
show
show_A


In [6]:
def show(func):
    print("show")
    if func.__name__=="test2":
        print("show_A")
        return func
    else:
        print("show_B")
        return int
# show(test)不會改變 test函式
# @show則會改變 test函式
show(test) 
def test(a):
    print("test")
    return a+5

show(test2)
def test2(a):
    print("test2")
    return a+3

# test(5) # test --> show_B --> show --> show_A
# print("---------")
# test2(5)

show
show_B
show
show_A


## [pandas.Series.apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html)

In [7]:
def poemcut(s):
    s = s.replace("\r\n","").replace("\r\n","")
    return " ".join(jieba.cut(s))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

0      日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1     朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2      李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3       故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4                 危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5                  床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6     天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
7               眾鳥 高 飛 儘 ， 孤雲獨 去 閒 。 相看 兩不厭 ， 隻 有 敬亭山 。
8     鳳凰 台上 鳳凰遊 ， 鳳去 台空江 自流 。 吳宮 花草 埋 幽徑 ， 晉代 衣冠 成古丘...
9     渡遠 荊門外 ， 來 從 楚國遊 。 山 隨 平野 儘 ， 江入 大荒 流 。 月 下 飛天...
10    百川 日東流 ， 客去 亦 不息 。 我 生苦 漂 蕩 ， 何時 有 終極 。 讚 公 釋 ...
11    細泉 兼 輕冰 ， 沮洳 棧道 濕 。 不辭 辛苦 行 ， 迫此 短景急 。 石門 雪雲隘 ...
12    首路 栗亭 西 ， 尚想 鳳凰村 。 季冬 攜 童稚 ， 辛苦 赴 蜀門 。 南登 木皮 嶺...
13    落日 在 簾 鉤 ， 溪邊 春事幽 。 芳菲 緣岸 圃 ， 樵 爨 倚灘 舟 。 啅 雀 爭...
14    竇侍 禦 ， 驥 之子 ， 鳳之雛 。 年 未 三十 忠義俱 ， 骨鯁 絕代 無 。 炯 如...
15    百草 競春華 ， 麗春應 最勝 。 少須 好 顏色 ， 多漫枝條 剩 。 紛紛 桃李 枝 ，...
16    何年 顧虎頭 ， 滿壁畫 瀛州 。 赤 日 石林 氣 ， 青天 江海 流 。 錫飛常 近鶴 ...
17    野寺 隱喬木 ， 山僧 高下 居 。 石門 日色異 ， 絳 氣橫 扶疏 。 窈窕 入

In [8]:
# 機器學習/深度學習的函式庫都要求進去要數字(不能是字串)
# value_counts(統計類別個數)
train_df['作者'].value_counts()

杜甫     1157
李白      969
白居易     605
Name: 作者, dtype: int64

In [9]:
# unique(不重複類別) 
train_df['作者'].unique()

array(['李白', '杜甫', '白居易'], dtype=object)

In [10]:
u = train_df['作者'].unique()
trans = {v:i for i,v in enumerate(u)}
reverse = {i:v for i,v in enumerate(u)}
trans
y_train = train_df['作者'].replace(trans)
y_test = test_df['作者'].replace(trans)
y_train

0       0
1       0
2       0
3       0
4       0
       ..
2726    2
2727    2
2728    2
2729    2
2730    2
Name: 作者, Length: 2731, dtype: int64

## [sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)

In [11]:
# {作者名稱:index}的字典
train_df['作者'] = train_df['作者'].astype('category')
saved_map = { cat:train_df['作者'].cat.categories.get_loc(cat) for cat in train_df['作者'].cat.categories }
saved_map

{'李白': 0, '杜甫': 1, '白居易': 2}

In [12]:
# (作者名稱)轉為整數(index)
train_df['作者'] = train_df['作者'].replace(saved_map)
test_df['作者'] = test_df['作者'].replace(saved_map)

[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

In [13]:
# 使用 TF-IDF 統計詞彙
# TF-IDF --> 對於(單純貝葉斯)沒有差, 因為(李白使用常用詞的機率)=(杜甫使用常用詞的機率),因此
# fit --> 只找出 min 與 max(測試資料集中,將會加入沒有看過的資料,因為推測該資料很重要)
# fit_transform --> 把每個格子依照 min,max 做出轉換(測試資料集中,將會捨棄沒有看過的資料,因為推測該資料不受影響)
# 訓練集: (fit)+(transform)
# 測試集: (transform)
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_vec = vec.fit_transform(x_train) # fit_transform = (fit)+(transform)
x_test_vec = vec.transform(x_test)
x_train_vec
print(x_train_vec)
# <2731x52294 sparse matrix of type '<class 'numpy.int64'>' with 85677 stored elements in Compressed Sparse Row format>
# 矩陣大小: 2731x52294, 有值的數量:85677
# 稀疏矩陣(sparse matrix) --> 

  (0, 16053)	1
  (0, 29006)	1
  (0, 30177)	1
  (0, 14002)	1
  (0, 139)	1
  (0, 4756)	1
  (0, 39345)	1
  (0, 51400)	1
  (0, 23289)	1
  (0, 25180)	1
  (0, 31382)	1
  (0, 3549)	1
  (0, 13846)	1
  (0, 50568)	1
  (0, 3797)	1
  (0, 26005)	1
  (0, 44987)	2
  (0, 34)	1
  (0, 33975)	1
  (1, 49338)	1
  (1, 16293)	1
  (1, 19165)	1
  (1, 4540)	1
  (1, 85)	1
  (1, 2776)	1
  :	:
  (2729, 8800)	1
  (2729, 6485)	1
  (2729, 3060)	1
  (2730, 25782)	1
  (2730, 6026)	1
  (2730, 36958)	1
  (2730, 15905)	1
  (2730, 44854)	1
  (2730, 29458)	1
  (2730, 2175)	1
  (2730, 31125)	1
  (2730, 40974)	1
  (2730, 35446)	1
  (2730, 49196)	1
  (2730, 6770)	1
  (2730, 48596)	1
  (2730, 7813)	1
  (2730, 1714)	1
  (2730, 39190)	1
  (2730, 28820)	1
  (2730, 38489)	1
  (2730, 28423)	1
  (2730, 51689)	1
  (2730, 8048)	1
  (2730, 23226)	1


In [14]:
word_to_index = vec.vocabulary_
index_to_word = {v:k for k,v in word_to_index.items()}
index_to_word[51400]

'高樓'

In [15]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vec = TfidfVectorizer()
# bag = vec.fit_transform(train_df['內容'])
# print("總共維度:", len(vec.get_feature_names()))

In [16]:
# 單純貝氏(分類)
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
# 開始預測
from sklearn.metrics import accuracy_score
predict = clf.predict(x_test_vec)
print("預測:", list(predict))
print("正確標籤:", list(y_test))
print("Naive-Bayes 正確率: ", accuracy_score(predict, y_test) * 100, "%")

預測: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Naive-Bayes 正確率:  80.0 %


In [18]:
from sklearn.metrics import confusion_matrix
i = [(name+"(正確)") for name in u]
c = [(name+"(預測)") for name in u]
pd.DataFrame(confusion_matrix(y_test,predict),
             index = i,
             columns = c)

,李白(預測),杜甫(預測),白居易(預測)
李白(正確),8,1,1
杜甫(正確),1,8,1
白居易(正確),1,1,8


In [20]:
# p = input("君不見 黃河之水")
p = "君不見 黃河之水"
p = poemcut(p) # 切割成(詞彙)
p = vec.transform([p])

proba = clf.predict_proba(p)[0] # 查看預測機率
for name,prob in zip(u,proba):
    print(name,"的機率: ",prob)
idx = clf.predict(p)[0]
print("應該是",u[idx],"寫的")

李白 的機率:  0.587110114075689
杜甫 的機率:  0.33123345596688514
白居易 的機率:  0.08165642995742552
應該是 李白 寫的


# 遷移學習:

## 單純貝葉斯種類:
* MultinomialNB: 特徵數量固定 --> 計算次數 --> 類似於擲骰子的機率
* GaussianNB: 以(不連續資料)來模擬(連續資料) --> $ P(Setosa)P(\frac{sl=4}{Setosa})P(\frac{sw=2}{Setosa}) $
* BernoulliNB:$ P(受歡迎)P(\frac{帥=1}{受歡迎}) $

## GaussianNB
$ \theta = \frac{N_{yi}+\alpha}{N_{y}+{\alpha}n} $ --> $ \alpha $可以避免發生機率為0(Laplace Smoothing=機率平滑化)

In [44]:
# KNN(分類)
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(bag, train_df['作者']) # 開始訓練

# 開始預測
predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['作者']))
print("kNN 正確率: ", accuracy_score(test_df['作者'], predict) * 100, "%")

預測: [2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
kNN 正確率:  36.666666666666664 %
